In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 下载训练集
train_dataset = datasets.MNIST(root='./',
                train=True,
                transform=transforms.ToTensor(),
                download=True)
# 下载测试集
test_dataset = datasets.MNIST(root='./',
               train=False,
               transform=transforms.ToTensor(),
               download=True)

In [3]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                      batch_size=batch_size,
                      shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset,
                     batch_size=batch_size,
                     shuffle=True)

In [4]:
for i, data in enumerate(train_loader):
    # 获得数据和对应的标签
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 全连接层
        self.fc1 = nn.Linear(784,10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # ([64, 1, 28, 28])->(64,784)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [6]:
LR = 0.5
# 定义模型
model = Net()
# 定义代价函数
mse_loss = nn.MSELoss()
# 定义优化器
optimizer = optim.SGD(model.parameters(), LR)

In [7]:
def train():
    for i, data in enumerate(train_loader):
        # 获得数据和对应的标签
        inputs, labels = data
        # 获得模型预测结果，（64，10）
        out = model(inputs)
        # to onehot,先把标签变成一个2维数据（64，1）
        labels = labels.reshape(-1,1)
        # 转独热编码
        # tensor.scatter_(dim, index, src) 
        # dim:对哪个维度进行独热编码
        # index:要将src中对应的值放到tensor的哪个位置。
        # src:插入index的数值
        one_hot = torch.zeros(inputs.shape[0], 10).scatter(1, labels, 1)
        # 计算loss，计算loss的两个数据的shape要一致
        loss = mse_loss(out, one_hot)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()


def test():
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得数据和对应的标签
        inputs, labels = data
        # 获得模型预测结果
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc: {0}".format(correct.item() / len(test_dataset)))

In [9]:
for epoch in range(0, 2):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test acc: 0.9098
epoch: 1
Test acc: 0.9134


In [15]:
torch.save(model.state_dict(), 'model/my_model.pth')